<a href="https://colab.research.google.com/github/slatej1/Everside/blob/main/Historic_Everside_Deal_Cloud_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import openpyxl

In [20]:

# UPDATE INPUT FILE NAME
# Read the data from the source file
source_file_path = 'Fund I DealCloud Model - Q3 2021.xlsx'
data = pd.read_excel(source_file_path, sheet_name='Overflow Fund', skiprows = 2)  # Adjust sheet_name as needed

# print(data.columns)

In [21]:
template_file_path = 'deal_cloud_dataset_template.xlsx'
workbook = openpyxl.load_workbook(template_file_path)
worksheet = workbook.active
# UPDATE OUTPUT FILE NAME
output_file_path = 'Direct I 2021 Q3.xlsx'

In [22]:
column_mapping = {
    'Company Name': 'E',
    'Geography': 'J',
    'Industry': 'I',
    'Date of Investment': 'H',
    'Debt': 'P',
    'Equity': 'Q',
    'Everside $ Expsoure': 'V',
    'Everside % Expsoure': 'W',
    'Debt (Cost)': 'Z',
    'Debt (FMV)': 'AB',
    'Equity (Cost)': 'AD',
    'Equity (FMV)': 'AF',
    'Cash ': 'AK',
    'PIK': 'AL',
    'Total Coupon': 'AM',
    'Cash': 'AP',
    'PIK.1': 'AQ',
    'Equity Onwership (FD)': 'AR',
    'Senior Debt': 'AS',
    'Total Debt': 'AU',
    'Liquidity (Cash + Available Liquidity)': 'AW',
    'FCCR (Covenants)': 'AX',
    'Enterprise Value': 'AY',
    'Sales': 'BA',
    'EBITDA': 'BB',
    'Capex': 'BC',
    'Senior Debt.1': 'BE',
    'Total Debt.1': 'BG',
    'Liquidity (Cash + Available Liquidity).1': 'BL',
    'FCCR (Current)': 'BM',
    'Enterprise Value.1': 'BN',
    'Sales.1': 'BP',
    'EBITDA.1': 'BQ',
    'Capex.1': 'BR',
}

In [23]:
def col_letter_to_num(letter):
    num = 0
    for char in letter:
        num = num * 26 + (ord(char.upper()) - ord('A') + 1)
    return num

In [24]:
start_row = 2

# Set the first three columns to "Pipeline"
for i in range(len(data)):
    for col_num in range(1, 2):
        worksheet.cell(row=start_row + i, column=col_num, value="Direct I")

for i in range(len(data)):
    for col_num in range(2, 4):
        worksheet.cell(row=start_row + i, column=col_num, value="Directs")

for i in range(len(data)):
    for col_num in range(4, 5):
        worksheet.cell(row=start_row + i, column=col_num, value="12/31/2021")

# Write data to the template based on the column mapping
for i, row in data.iterrows():
    for source_col, target_cols in column_mapping.items():
        if isinstance(target_cols, list):
            # Handle mapping a single source column to multiple target columns
            for target_col in target_cols:
                value = row[source_col]
                col_num = col_letter_to_num(target_col)  # Convert column letter to number
                worksheet.cell(row=start_row + i, column=col_num, value=value)
        elif isinstance(target_cols, dict) and 'sources' in target_cols:
            # Handle combined or subtracted columns
            sources = target_cols['sources']
            target_col_letter = target_cols['target']
            if source_col.startswith('Subtracted'):
                # Subtract the sum of specified source columns
                value = row[sources[0]] - sum(row[source] for source in sources[1:])
            else:
                # Combine specified columns
                value = sum(row[source] for source in sources)
            col_num = col_letter_to_num(target_col_letter)  # Convert column letter to number
            worksheet.cell(row=start_row + i, column=col_num, value=value)
        else:
            # Single column mapping
            value = row[source_col]
            col_num = col_letter_to_num(target_cols)  # Convert column letter to number
            worksheet.cell(row=start_row + i, column=col_num, value=value)

In [25]:
# Save the updated template
workbook.save(output_file_path)